In [10]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from df_plot import plot_df 
from info_value_to_net import info_value_to_net
from get_info_values import mutual_info_measures, cond_kl_divergence
from save_info_values import save_info_values

from preprocessing import preprocessing

np.seterr(divide='ignore', invalid = 'ignore')

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

### Designing a national strategy

#### Option 1: Perform FIT to the n patients with higher utility measured by the model, being n the number of people that the current screening strategy covers (that is, the number of people above the age of 54)

In [104]:
from preprocessing import preprocessing


try: 
    df_test = pd.read_csv("private/df_2016.csv")
    df_test = preprocessing(df_test)

    df_test = df_test.rename(columns = {"Hyperchol.": "Hyperchol_"})

    df_utilities_2016 = pd.read_csv("utilities_2016.csv")

except:
    model_type = "linear"

    net = pysmile.Network()
    net.read_file(f"decision_models/DM_screening_rel_pcmi_{model_type}.xdsl")

    df_test = pd.read_csv("private/df_2016.csv")
    df_test = preprocessing(df_test)

    df_test = df_test.rename(columns = {"Hyperchol.": "Hyperchol_"})

    y = np.array(list(df_test["CRC"]*1))

    vars1 = np.array(["No scr", "gFOBT", "FIT", "Blood_test", "sDNA", "CTC", "CC", "Colonoscopy"])

    df_utilities_2016 = pd.DataFrame([], columns = vars1)

    for i in range(df_test.shape[0]):
        sample = df_test.iloc[i].drop(labels = ["CRC"])
        sample_dict = sample.to_dict() 

        net.clear_all_evidence()

        for key, value in sample.items():
            if type(value) == np.bool_:
                net.set_evidence(key, bool(value))
            else:
                net.set_evidence(key, value)

        net.update_beliefs()

        arr = np.array(net.get_node_value("U"))

        U = np.concatenate((arr[::2], [arr[1]]), axis = 0)

        row = pd.DataFrame(U.reshape(1,8), columns=vars1)
        df_utilities_2016 = pd.concat([df_utilities_2016, row], ignore_index = True)

    df_utilities_2016 = pd.concat( [df_utilities_2016, df_test["CRC"] ] , axis = 1)
    df_utilities_2016.to_csv("utilities_2016.csv", index = False)


In [105]:
df_selected_current_strategy = df_test[df_test["Age"] == "age_5_old_adult"]
tot_num_patients = df_selected_current_strategy.shape[0]   
print("------------------------")
print("Number of selected patients by current strategy is: ", tot_num_patients)
print("Number of patients with CRC by current strategy is: ", df_selected_current_strategy["CRC"].sum())
print("Percentage of total patients with CRC detected:",  (df_selected_current_strategy["CRC"].sum() / df_test["CRC"].sum()).round(4) * 100, "%")

print("------------------------")
print("Selecting the same number of patients for the new strategy, that is, ", df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients].shape[0])
print("Number of patentes with CRC by new strategy is: ", df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum())
print("Percentage of total patients with CRC detected:", (df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum() / df_test["CRC"].sum()).round(4) * 100, "%")

print("------------------------")
print("This is an improvement of ", df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum() - df_selected_current_strategy["CRC"].sum(), "patients")
print("A proportional improvement of ", ((df_utilities_2016.sort_values(by = "FIT", ascending = False)[:tot_num_patients]["CRC"].sum() - df_selected_current_strategy["CRC"].sum()) / df_selected_current_strategy["CRC"].sum()).round(4) * 100, "%")

------------------------
Number of selected patients by current strategy is:  49074
Number of patients with CRC by current strategy is:  107
Percentage of total patients with CRC detected: 49.08 %
------------------------
Selecting the same number of patients for the new strategy, that is,  49074
Number of patentes with CRC by new strategy is:  109
Percentage of total patients with CRC detected: 50.0 %
------------------------
This is an improvement of  2 patients
A proportional improvement of  1.87 %


#### Option 2: Apply new national strategy with N FITs and M colonoscopies available.

In [5]:
N_FIT = 2000
M_COL = 600

In [96]:
import numpy as np
import pandas as pd

def simulate_test_results(sensitivity_scr, specificity_scr, cost_scr,
                           sensitivity_col, specificity_col, cost_col,  patient_data):
    """
    Simulate test results based on sensitivity, specificity, and actual number of patients
    with and without the disease.
    
    Parameters:
    - sensitivity (float): Sensitivity of the test (true positive rate)
    - specificity (float): Specificity of the test (true negative rate)
    - num_with_disease (int): Number of patients who have the disease
    - num_without_disease (int): Number of patients who do not have the disease
    
    Returns:
    - pandas DataFrame: A DataFrame with the simulated test results, true conditions, and test outcomes.
    """
    print("Number of patients considered: ", patient_data.shape[0])

    num_with_disease = patient_data.sum()
    num_without_disease = len(patient_data) - num_with_disease

    # Step 1: Create a list of patients with and without the disease
    conditions = patient_data

    print(f"Cost of screening: {cost_scr*patient_data.shape[0]} €")
    
    # Step 2: Simulate test results
    scr_results = []
    col_results = []
    
    for condition in conditions:
        if condition == 1:
            # Patient has the disease, test is positive with probability = sensitivity
            scr_result = np.random.choice([1, 0], p=[sensitivity_scr, 1 - sensitivity_scr])
            
        else:
            # Patient does not have the disease, test is negative with probability = specificity
            scr_result = np.random.choice([0, 1], p=[specificity_scr, 1 - specificity_scr])
            col_result = np.random.choice([0, 1], p=[specificity_col, 1 - specificity_col])

        scr_results.append(scr_result)
        col_results.append(col_result)
    
    # Step 3: Create a DataFrame to store the results
    df_scr = pd.DataFrame({
        'Condition': conditions,    # True condition of the patient
        'TestResult': scr_results  # Simulated test result
    })

    # Add columns to indicate true positives, false positives, etc.
    df_scr['TruePositive'] = (df_scr['Condition'] == 1) & (df_scr['TestResult'] == 1)
    df_scr['FalsePositive'] = (df_scr['Condition'] == 0) & (df_scr['TestResult'] == 1)
    df_scr['TrueNegative'] = (df_scr['Condition'] == 0) & (df_scr['TestResult'] == 0)
    df_scr['FalseNegative'] = (df_scr['Condition'] == 1) & (df_scr['TestResult'] == 0)
    
    # Step 4: Calculate confusion matrix components
    TP = df_scr['TruePositive'].sum()
    FP = df_scr['FalsePositive'].sum()
    TN = df_scr['TrueNegative'].sum()
    FN = df_scr['FalseNegative'].sum()
    
    # Create confusion matrix
    confusion_matrix_scr = pd.DataFrame({
        'Predicted Negative': [TN, FN],
        'Predicted Positive': [FP, TP]
    }, index=['Actual Negative', 'Actual Positive'])


    # For FIT positives, perform colonoscopy:
    FIT_positives = df_scr[df_scr["TestResult"] == 1]

    print("Number of FIT positives: ", FIT_positives.shape[0])
    print("Number of colonoscopies to be done: ", FIT_positives.shape[0])

    print(f"Cost of colonoscopy program: {cost_col*FIT_positives.shape[0]} €")


    conditions = FIT_positives["Condition"].to_list()

    col_results = []

    for condition in conditions:
        if condition == 1:
            # Patient has the disease, test is positive with probability = sensitivity
            col_result = np.random.choice([1, 0], p=[sensitivity_col, 1 - sensitivity_col])
        else:
            # Patient does not have the disease, test is negative with probability = specificity
            col_result = np.random.choice([0, 1], p=[specificity_col, 1 - specificity_col])

        col_results.append(col_result)

    # Step 5: Create a DataFrame to store the results
    df_col = pd.DataFrame({
        'Condition': conditions,    # True condition of the patient
        'TestResult': col_results  # Simulated test result
    })

    # Add columns to indicate true positives, false positives, etc.
    df_col['TruePositive'] = (df_col['Condition'] == 1) & (df_col['TestResult'] == 1)
    df_col['FalsePositive'] = (df_col['Condition'] == 0) & (df_col['TestResult'] == 1)
    df_col['TrueNegative'] = (df_col['Condition'] == 0) & (df_col['TestResult'] == 0)
    df_col['FalseNegative'] = (df_col['Condition'] == 1) & (df_col['TestResult'] == 0)

    # Step 6: Calculate confusion matrix components
    TP = df_col['TruePositive'].sum()
    FP = df_col['FalsePositive'].sum()
    TN = df_col['TrueNegative'].sum()
    FN = df_col['FalseNegative'].sum()

    # Create confusion matrix
    confusion_matrix_col = pd.DataFrame({
        'Predicted Negative': [TN, FN],
        'Predicted Positive': [FP, TP]
    }, index=['Actual Negative', 'Actual Positive'])

    print("Number of CRC true positive cases detected by colonoscopy: ", TP)
    print("Number of false positives by colonoscopy: ", FP)

    print(f"Total cost of screening and colonoscopy: {cost_scr*patient_data.shape[0] + cost_col*FIT_positives.shape[0]} €")
    print("Proportion of total CRC cases in the whole population detected by the method: ", TP / df_test["CRC"].sum())
    print("Proportion of cases in the high-risk target population detected by the method: ", TP / y.sum())
    
    return df_scr, confusion_matrix_scr, df_col, confusion_matrix_col

**Old screening strategy**

In [97]:
df_test = pd.read_csv("private/df_2016.csv")
df_test = preprocessing(df_test)

df_test = df_test.rename(columns = {"Hyperchol.": "Hyperchol_"})

In [99]:
sensitivity_scr = 0.75
specificity_scr = 0.966
cost_scr = 14.34

sensitivity_col = 0.95
specificity_col = 0.99
cost_col = 1000

# Select old adults as high-risk target population
y = np.array(list(df_test[df_test["Age"] == "age_5_old_adult"]["CRC"]*1))

df_scr, confusion_matrix_scr, df_col, confusion_matrix_col = simulate_test_results(sensitivity_scr, specificity_scr, cost_scr, 
                                                                                    sensitivity_col, specificity_col, cost_col, y)

Number of patients considered:  49074
Cost of screening: 703721.16 €
Number of FIT positives:  1768
Number of colonoscopies to be done:  1768
Cost of colonoscopy program: 1768000 €
Number of CRC true positive cases detected by colonoscopy:  70
Number of false positives by colonoscopy:  21
Total cost of screening and colonoscopy: 2471721.16 €
Proportion of total CRC cases in the whole population detected by the method:  0.3211009174311927
Proportion of cases in the high-risk target population detected by the method:  0.6542056074766355


**New screening strategy**

Say we have 10,000 FITs and 2,000 colonoscopies. We would take the first 2,000 patients with highest utility based on our model and perform FIT on them. If positive, then proceed with a colonoscopy.

In [83]:
df_utilities_2016 = pd.read_csv("utilities_2016.csv")
df_utilities_2016_sorted = df_utilities_2016.sort_values(by = "FIT", ascending = False)

In [103]:
sensitivity_scr = 0.75
specificity_scr = 0.966
cost_scr = 14.34

sensitivity_col = 0.95
specificity_col = 0.99
cost_col = 1000

# Select as high-risk target population the top 20000 patients according to utility in our model
y = np.array(list(df_utilities_2016_sorted["CRC"]*1))[:49074]

df_scr, confusion_matrix_scr, df_col, confusion_matrix_col = simulate_test_results(sensitivity_scr, specificity_scr, cost_scr, sensitivity_col, specificity_col, cost_col, y)

Number of patients considered:  49074
Cost of screening: 703721.16 €
Number of FIT positives:  1695
Number of colonoscopies to be done:  1695
Cost of colonoscopy program: 1695000 €
Number of CRC true positive cases detected by colonoscopy:  77
Number of false positives by colonoscopy:  14
Total cost of screening and colonoscopy: 2398721.16 €
Proportion of total CRC cases in the whole population detected by the method:  0.3532110091743119
Proportion of cases in the high-risk target population detected by the method:  0.7064220183486238
